## Setting up AWS Athena


### Step 1: Go to AWS Glue Service to Create and Run a "Crawler"

* A crawler is an automated service in AWS that looks through a set of classifiers prioritized by the schema to create metadata tables.  

* To get the most recent version of the metadata, which is updated by SRA daily, it is necessary to rerun the crawler.

* This differs from BigQuery because BigQuery database and table names are static and the metadata tables are recreated daily by SRA using automated scripts.

<img src="./img/crawler.png" align="left" width="700"/>

### Step 2: Confirm the Name of the Database and the Tables

* When the crawler is created, the user sets the database name. In this workshop, "sra" was the name set by the crawler.

* The level of the tables that show up in the database depends on which datastore is targeted by the crawler. In this case, all the SRA tables are visible because the crawler was set to run on the highest level directory **s3://sra-pub-metadata-us-east-1**

* Note that the tables are all in one database called **"sra"**. This differs from BigQuery which has two sets of tables in its schema: **"sra"** and **"sra_tax_analysis_tool"**


#### Athena Tables

<img src="./img/table.png" align="center" width="700"/>





#### BigQuery Tables

<img src="./img/BigQuery.png" align="center" width="350"/>


The Query workspace for Athena looks very similar to the BigQuery workspace.  The syntax is almost identical, but before starting, make note of how the Database and Tables are structured. The column names are exactly the same between the two providers.

<img src="./img/athena.png" align="center" width="800"/>

### Step 3: Create a "Staging" Bucket

* Athena differs from BigQuery in that it requires a user to create a bucket to hold all the metadata and query outputs

* This is true for both command line interface as well as direct console queries.

### Run the following command to create your bucket for this workshop

This script also applies credentials that allow this notebook (which is on GCP) to access AWS to create buckets and run Athena. It also creates a connection using sqlalchemy to the Athena database.

*The script is unique to this workshop, so make sure to add your own AWS credentials when working outside of this event. Refer to the setup_AWS_credentials.ipynb notebook for tips and tricks on how to do this.*

In [128]:
%run setup_AWS_credentials.py

arhodes-sql-workshop
/arhodes-sql-workshop
BUCKETS	2021-08-30T19:29:32.000Z	arhodes-sql-workshop
Done.


### Test that the Credentials were Loaded Correctly

In [127]:
%%sql

SELECT acc, bioproject 
FROM sra.metadata
WHERE organism = 'Homo sapiens'
LIMIT 5



Done.


acc,bioproject
ERR868965,PRJEB9309
ERR452072,PRJEB5480
ERR452433,PRJEB5480
ERR452651,PRJEB5480
ERR453061,PRJEB5480


# Problem : I want to find RNA-Seq data from estrogen receptor negative and progesterone positive breast cancer cell lines.


Hormone receptor positive (HR+) breast cancer cells may have either estrogen receptors (ER+) or progesterone receptors (PR+) or both.

We have already found one search strategy to find a dataset that had ER+ breast cancer cell lines.

This case study will look for HR+ breast cancer cell lines that are ER- and PR+

This will requre combining search strategies to narrow our choices.

These searchers are taking place on Athena, so the syntax is mostly the same with some subtle differences.

Some of they syntax, such as capitalization or lowercase, is dicated by the tool we are using, so if a query in this notebook does not work exactly the same in Athena, try that first.

### Start with a more general search that uses an "or" to specify receptor types

In [138]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')


Done.


acc,bioproject,k,v
SRR10415374,PRJNA588284,primary_search,02-MaP-WS-TER-DMS1_S4_L001_R1_001.fastq.gz
ERR3580192,PRJEB27236,description_sam,"Protocols: ER-alpha positive BC cells: MCF-7 (ATCC HTB-22),were propagated in Dulbecco's modified Eagle medium (DMEM; Sigma-Aldrich) supplemented with 10% FBS (HyClone) and antibiotics: 100 U/ml penicillin, 100 mg/ml streptomycin, 250 ng/ml amphoterici..."
SRR7764009,PRJNA488315,ecotype_sam,LER-1
ERR1096791,PRJEB11517,description_sam,"HAP1 FER-KO cells, stimulated with RESV, replicate R1"
ERR1096791,PRJEB11517,title_sam,"HAP1 FER-KO cells, stimulated with RESV, replicate R1"
ERR2733144,PRJEB28170,primary_search,ena-EXPERIMENT-IFREMER-10-08-2018-12:16:30:787-1
ERR2733144,PRJEB28170,primary_search,ena-RUN-IFREMER-10-08-2018-12:16:30:787-1
ERR2733144,PRJEB28170,primary_search,ena-STUDY-IFREMER-10-08-2018-12:16:30:953-292
ERR3063014,PRJEB20879,primary_search,ena-EXPERIMENT-IFREMER-15-01-2019-10:06:13:917-2
ERR3063014,PRJEB20879,primary_search,ena-RUN-IFREMER-15-01-2019-10:06:13:917-2


### A lot of the metadata we are picking up is not related to samples. So the next search specifies that the key value needs to end in "sam" by using a wildcard "%" before "sam" and not after.

In [139]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE '%sam'





Done.


acc,bioproject,k,v
SRR7404561,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404497,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404538,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7431046,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7430970,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404662,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404670,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404606,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7404566,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER
SRR7431011,PRJNA416091,cre_line_sam,Col1a2:ER-Cre-ER


### Adding the word "receptor" in front of the wildcard here may limit the samples to a reasonable set for the next steps. 

There are several strategies to find interesting data sets, this is one that seems to work just within this case study. A good practice would be to spend some time trying different methods to get the best data set for your research question.

In [160]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'





Done.


acc,bioproject,k,v
SRR12060808,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060807,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060825,PRJNA640820,receptor_status_sam,HR+
SRR12060827,PRJNA640820,receptor_status_sam,HR+
SRR12060826,PRJNA640820,receptor_status_sam,HR+
SRR12060824,PRJNA640820,receptor_status_sam,HR+
SRR12060809,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060806,PRJNA640820,receptor_status_sam,ER-/PR+


#### This looks like a good set, let's pull out the accession list for downstream analysis.

The syntax is slightly different because we are using sqlmagic - a different program than bigquery.

The output of a query can be saved as a results object which is easily made into a dataframe for downstream analysis.  We are creating a results object called "accessions" and then saving it into a text file called "cell_line_ER_HR.txt"



In [143]:
%%sql accessions <<


SELECT acc
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'

Done.
Returning data to local variable accessions


#### The output is different in SQL magic compared to BigQuery magic in the other notebook.

It is not a dataframe yet, it would need to have accessions.DataFrame() to show the index as in the BigQuery results object.

In [144]:
accessions

acc
SRR12060809
SRR12060806
SRR12060824
SRR12060808
SRR12060807
SRR12060825
SRR12060826
SRR12060827


In [147]:
accessions.DataFrame()

,acc
0,SRR12060809
1,SRR12060806
2,SRR12060824
3,SRR12060808
4,SRR12060807
5,SRR12060825
6,SRR12060826
7,SRR12060827


In [146]:
file = open("cell_line_ER_HR.txt", "w")
file.write(accessions.DataFrame().to_string(index=False, header=False))
file.close()

***
***

### We may not have time to go over this in detail, the code is here for your information ###

<img src="./img/Workflow_demo.png" align="left" width="700"/>




### We are going to skip down to the final analysis for this demo  ###

In [150]:
%%bash

cat cell_line_ER_HR.txt

SRR12060809
SRR12060806
SRR12060824
SRR12060808
SRR12060807
SRR12060825
SRR12060826
SRR12060827

In [151]:
%%bash

cat cell_line_ER_HR.txt | xargs -P 12 -n 1 fastq-dump -X 1000000 -O cell_line_fastq

Read 1000000 spots for SRR12060824
Written 1000000 spots for SRR12060824
Read 1000000 spots for SRR12060826
Written 1000000 spots for SRR12060826
Read 1000000 spots for SRR12060809
Written 1000000 spots for SRR12060809
Read 1000000 spots for SRR12060808
Written 1000000 spots for SRR12060808
Read 1000000 spots for SRR12060825
Written 1000000 spots for SRR12060825
Read 1000000 spots for SRR12060827
Written 1000000 spots for SRR12060827
Read 1000000 spots for SRR12060806
Written 1000000 spots for SRR12060806
Read 1000000 spots for SRR12060807
Written 1000000 spots for SRR12060807


In [152]:
%%bash

ls cell_line_fastq

SRR12060806.fastq
SRR12060807.fastq
SRR12060808.fastq
SRR12060809.fastq
SRR12060824.fastq
SRR12060825.fastq
SRR12060826.fastq
SRR12060827.fastq


In [153]:
%%bash

datasets download genome accession GCF_000001405.39 --chromosomes 17 --include-gtf --exclude-protein --exclude-rna



Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.88kB 14.7MB/s
Downloading: ncbi_dataset.zip    1.98kB 16.9kB/s
Downloading: ncbi_dataset.zip    1.98kB 16.9kB/s
Downloading: ncbi_dataset.zip    1.98kB 16.9kB/s
Downloading: ncbi_dataset.zip    32.8kB 220kB/s
Downloading: ncbi_dataset.zip    32.8kB 220kB/s
Downloading: ncbi_dataset.zip    65.5kB 373kB/s
Downloading: ncbi_dataset.zip    65.5kB 373kB/s
Downloading: ncbi_dataset.zip    65.5kB 373kB/s
Downloading: ncbi_dataset.zip    164kB 806kB/s
Downloading: ncbi_dataset.z

New version of client (12.12.0) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets


In [154]:
%%bash

unzip -o ncbi_dataset.zip -d GRCh38.p13

Archive:  ncbi_dataset.zip
  inflating: GRCh38.p13/README.md    
  inflating: GRCh38.p13/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.unlocalized.scaf.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/cds_from_genomic.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/genomic.gff  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/genomic.gtf  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/sequence_report.jsonl  
  inflating: GRCh38.p13/ncbi_dataset/data/dataset_catalog.json  


In [155]:
%%bash

makeblastdb -in GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna -dbtype nucl -parse_seqids -out GRCh38.p13/GRCh38.p13





Building a new DB, current time: 09/15/2021 08:47:31
New DB name:   /home/jupyter-arhodes/ASHG-Workshop-2021/GRCh38.p13/GRCh38.p13
New DB title:  GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 1.05871 seconds.




In [156]:
%%bash

mkdir cell_line_bams

### Making alignments

This script can be automated, but for the purposes of this demo, we are choosing only two of the fastq files to align to Chromosome 17 - a hot spot of activity for HER genes.  

* SRR12060806 is HR+

* SRR12060824 is ER-/PR+

Doing a little more digging into bioproject PRJNA640820: "62 breast cell lines composed of 27 Triple negative breast cancers (TNBC), 5 Non malignant (NM) lines, 14 Hormone receptor positive (HR+) and 16 Her2 amplified lines (Her2amp) were profiled." This was a relatively recent dataset produced on a NextSeq here in Boston.

<img src="./img/citation.png" align="left" width="700"/>


https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE152908



In [158]:
%%bash

magicblast -query cell_line_fastq/SRR12060806.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060806.bam -O BAM

[bam_sort_core] merging from 0 files and 8 in-memory blocks...


In [159]:
%%bash

magicblast -query cell_line_fastq/SRR12060824.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060824.bam -O BAM

[bam_sort_core] merging from 0 files and 8 in-memory blocks...


In [161]:
%%bash

samtools index cell_line_bams/SRR12060824.bam
samtools index cell_line_bams/SRR12060806.bam

### Load the Data in the NCBI Genome Data Viewer

These bam files with their indexes need to be loaded somewhere they can be retrieved by the NCBI Genome Data Viewer.

If you want to load your own remote files into GDV Viewer, it is possible using your Github repository for files that are **less than 1 GB**. Just use the following file format based on the location of your file in the repository.


Follow this format:  https:///<i></i>github.com/YOUR-GITHUB-ACCOUNT/YOUR-GITHUB-REPOSITORY/raw/master/PATH-TO-BAM



## See the Two Bams from this Demo

The two aligned bams have been pre-loaded at the following link: 
    

https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?cfg=NCID_1_53497080_130.14.18.128_9146_1631700092_3532671673

which will be good for the next 90 days.






Here are the links to the two bams if you want to load them again, make sure to zoom to Chromosome 17, that is where the reads were aligned. The screenshot below indicates where to enter these URL's to have the tracks show up.


<img src="./img/add_remote_view.png" align="left" width="600"/>


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060806.bam


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060824.bam

## What did we find???

Actually, starting with a general idea and targeting a specific question but being general in our search terms, this demo shows the power of using SQL to rapidly search millions of records in SRA. 

The speed and specificity of using SQL for will help researchers quickly locate sequences of interest, bring these sequences directly into a workflow and compare the outcomes within the command line interface.



***

#### The GDV link leads to the TP53 gene, an important protein in the development of cancer.



<img src="./img/tp53_gdv.png" align="left" width="600"/>



***

#### The mismatch in the pileups help narrow the scope of where interesting SNPs might be

<img src="./img/pileup_view.png" align="left" width="600"/>



***

#### One of the cell lines has a SNP that indicates that cisplatin will be a less effective drug treatment.


<img src="./img/drug_response_variant.png" align="left" width="600"/>






***

#### Reading more into the records shows the importance of this variant for many cancer drugs.

<img src="./img/clinvar.png" align="left" width="600"/>

The cell line with the SNP may be a good candidate for drug therapy studies to emulate individuals whose cancers do not respond to cisplatin. 